In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
sc = SparkContext(appName="PythonStreamingNetworkWordCount")
ssc = StreamingContext(sc, 1)

lines = ssc.socketTextStream("localhost", 9092)
counts = lines.flatMap(lambda line: line.split(" "))\
              .map(lambda word: (word, 1))\
              .reduceByKey(lambda a, b: a+b)
counts.pprint()

ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2020-07-09 16:02:05
-------------------------------------------

-------------------------------------------
Time: 2020-07-09 16:02:06
-------------------------------------------

-------------------------------------------
Time: 2020-07-09 16:02:07
-------------------------------------------

-------------------------------------------
Time: 2020-07-09 16:02:08
-------------------------------------------

-------------------------------------------
Time: 2020-07-09 16:02:09
-------------------------------------------

-------------------------------------------
Time: 2020-07-09 16:02:10
-------------------------------------------

-------------------------------------------
Time: 2020-07-09 16:02:11
-------------------------------------------

-------------------------------------------
Time: 2020-07-09 16:02:12
-------------------------------------------

-------------------------------------------
Time: 2020-07-09 16:02:13
----------

In [56]:
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import requests
import datetime
from pyspark.sql.functions import col, unix_timestamp, to_timestamp
spark = SparkSession.builder.appName('covid').getOrCreate()

In [82]:
def parse_date(text):
    for fmt in ('%-d/%-m/%Y %H:%M', '%d/%m/%Y', '%d/%m/%Y %H:%M', '%d/%m/%Y %H.%M', '%d/%m/%Y %-I.%M'):
        try:
            return datetime.datetime.strptime(text.strip(), fmt)
        except ValueError:
            pass
    print(text)
    raise ValueError('no valid date format found')

url = "https://api.covid19india.org/travel_history.json"
fields = [
    StructField('pid',StringType(), True),
    StructField('address',StringType(), True),
    StructField('timefrom',StringType(), True),
    StructField('timeto',StringType(), True)
]
schema = StructType(fields=fields)


response = requests.get(url)
dataList = [] 
for i in response.json()['travel_history']:
    tDict = { key : i[key] for key in ['pid','address', 'timefrom', 'timeto']}
#     tDict['timefrom'] = parse_date(tDict['timefrom'][0:16])
    dataList.append(tDict)

df = spark.createDataFrame(dataList,schema)
df.filter("pid == 'P35'").sort("timefrom").show()
df.withColumn(
    'new_date',
    unix_timestamp(col('timefrom'), "dd/mm/yyyy HH:mm:ss").cast("timestamp")
).sort('new_date').show(truncate=False)

# df.withColumn('converted_date',to_timestamp(col('timefrom'), "dd/mm/yyyy HH:mm:ss")).show(2)

03/082020


ValueError: no valid date format found

In [4]:
import sys
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages=org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell"

sc = SparkContext(appName="PythonStreamingRecieverKafkaWordCount")
ssc = StreamingContext(sc, 2) 
broker, topic = "localhost:9092", "test-topic"
kvs = KafkaUtils.createStream(ssc, \
                              broker, \
                              "raw-event-streaming-consumer",{topic:1}) 
lines = kvs.map(lambda x: x[1])
counts = lines.flatMap(lambda line: line.split(" ")) \
              .map(lambda word: (word, 1)) \
              .reduceByKey(lambda a, b: a+b)
counts.pprint()
ssc.start()
ssc.awaitTermination()

ModuleNotFoundError: No module named 'pyspark.streaming.kafka'

In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages=org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell"

conf = SparkConf()
conf.set('spark.jars', '/usr/local/spark/jars/spark-sql-kafka-0-10_2.12-3.0.0.jar')
conf.set("spark.sql.streaming.checkpointLocation","/tmp/lijo/checkpoint");


spark = SparkSession.builder \
        .config(conf=conf) \
        .appName('kafka-stream') \
        .getOrCreate()

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "test-topic") \
  .load()
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

ds = df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("console") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("topic", "test-topic") \
  .start()

ds.awaitTermination()


KeyboardInterrupt: 